# Reproducing Med-PaLM M: An Open-Source Approach to Generalist Biomedical AI

**Paper:** Tu et al., "Towards Generalist Biomedical AI" ([arXiv:2307.14334](https://arxiv.org/abs/2307.14334))

**Author:** MD Rabbi · Department of Computer Science · February 2026

This notebook reproduces key experiments from Google's Med-PaLM M paper using open-source models.
We replace PaLM-E (562B) with BLIP-2 (3.4B) and implement the paper's complete medical VQA pipeline.

⚠️ **Before running:** Go to **Runtime → Change runtime type → T4 GPU**

⏱️ **Total time:** ~1 hour

---

In [ ]:
#@title **Setup: GPU Check, Clone Repo, Install Dependencies** (run this first, ~3 min)
import torch, os, json

# GPU check
assert torch.cuda.is_available(), "❌ No GPU! Go to Runtime → Change runtime type → T4 GPU"
gpu_name = torch.cuda.get_device_name(0)
vram = torch.cuda.get_device_properties(0).total_memory / 1e9
print(f"✅ GPU: {gpu_name} ({vram:.1f} GB VRAM)")

# Clone repo
!rm -rf biomed-multimodal-reproduction
!git clone -q https://github.com/Mrabbi3/biomed-multimodal-reproduction.git
%cd biomed-multimodal-reproduction
print("✅ Repository cloned")

# Install dependencies
!pip install -q transformers>=4.36.0 accelerate>=0.25.0 peft>=0.7.0 \
    bitsandbytes datasets Pillow tqdm pyyaml nltk rouge-score matplotlib seaborn evaluate
print("✅ Dependencies installed")

# Patch BLIP-2 wrapper for newer transformers (BitsAndBytesConfig instead of load_in_8bit kwarg)
patch = '''"""BLIP-2 Model Wrapper for Medical VQA."""
import torch
from PIL import Image
from .base_model import BaseBiomedModel

class BLIP2Wrapper(BaseBiomedModel):
    def __init__(self, model_name="Salesforce/blip2-flan-t5-xl", device=None, load_in_8bit=False):
        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        super().__init__(model_name, device)
        self.load_in_8bit = load_in_8bit

    def load_model(self):
        from transformers import Blip2Processor, Blip2ForConditionalGeneration
        print(f"Loading BLIP-2: {self.model_name}")
        self.processor = Blip2Processor.from_pretrained(self.model_name)
        load_kwargs = {}
        if self.load_in_8bit and self.device == "cuda":
            from transformers import BitsAndBytesConfig
            load_kwargs["quantization_config"] = BitsAndBytesConfig(load_in_8bit=True)
            load_kwargs["device_map"] = "auto"
        else:
            load_kwargs["torch_dtype"] = torch.float16 if self.device == "cuda" else torch.float32
        self.model = Blip2ForConditionalGeneration.from_pretrained(self.model_name, **load_kwargs)
        if not self.load_in_8bit and self.device == "cuda":
            self.model = self.model.to(self.device)
        self.model.eval()
        param_count = sum(p.numel() for p in self.model.parameters()) / 1e9
        print(f"Model loaded ({param_count:.1f}B parameters)")

    def generate(self, image, prompt, max_new_tokens=256, temperature=1.0, num_beams=5):
        if self.model is None: raise RuntimeError("Call load_model() first")
        if image.mode != "RGB": image = image.convert("RGB")
        inputs = self.processor(images=image, text=prompt, return_tensors="pt")
        if not self.load_in_8bit:
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
        else:
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=temperature, num_beams=num_beams)
        return self.processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    def generate_batch(self, images, prompts, max_new_tokens=256):
        if self.model is None: raise RuntimeError("Call load_model() first")
        rgb_images = [img.convert("RGB") if img.mode != "RGB" else img for img in images]
        inputs = self.processor(images=rgb_images, text=prompts, return_tensors="pt", padding=True)
        if not self.load_in_8bit:
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
        else:
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model.generate(**inputs, max_new_tokens=max_new_tokens)
        return [t.strip() for t in self.processor.batch_decode(outputs, skip_special_tokens=True)]
'''
with open('models/blip2_wrapper.py', 'w') as f:
    f.write(patch)
print("✅ BLIP-2 wrapper patched for Colab")
print("\n🚀 Setup complete! Run cells below in order.")

---
## Phase 1: Download Data & Sanity Check (~2 min)

In [ ]:
#@title **Phase 1: Download VQA-RAD + Run Sanity Checks**
!python data/download.py --dataset vqa_rad
print("---")
!python experiments/01_data_sanity_check.py

In [ ]:
#@title **Visualize: Sample Medical Images from VQA-RAD**
import os
from IPython.display import Image, display
fig_path = "results/figures/vqa_rad_sanity_check.png"
if os.path.exists(fig_path):
    display(Image(filename=fig_path, width=800))
    print("✅ Data loaded correctly — images match their questions")
else:
    print("⚠️ Image not generated")

---
## Phase 2: Zero-Shot Baseline (~10 min)

Load BLIP-2 and test on VQA-RAD **without any training**.
This downloads the 15GB model on first run. Be patient.

In [ ]:
#@title **Phase 2: Zero-Shot Forward Pass (downloads ~15GB model)**
!python experiments/02_forward_pass_test.py --model blip2 --max_samples 50 --quantize

In [ ]:
#@title **View: Zero-Shot Baseline Results**
import os, json
path = "results/tables/baseline_metrics.json"
if os.path.exists(path):
    with open(path) as f:
        b = json.load(f)
    print("ZERO-SHOT BASELINE (no training)")
    print("=" * 50)
    print(f"  Our BLIP-2 (3.4B):     BLEU-1 = {b['bleu_1']:.2f}%  |  F1 = {b['f1']:.2f}%")
    print(f"  PaLM-E 84B (paper):    BLEU-1 = 59.19%  |  F1 = 38.67%")
    print(f"  Med-PaLM M 562B:       BLEU-1 = 71.27%  |  F1 = 62.06%")
    print(f"\n  Gap explained by 165x parameter difference (3.4B vs 562B)")
else:
    print("⚠️ Run Phase 2 first")

---
## Phase 3: Overfit Test — Verify Training Pipeline (~5 min)

Train on 5 examples for 50 epochs. If the model can memorize them, the pipeline works.

In [ ]:
#@title **Phase 3: Overfit 5 Examples (Training Pipeline Validation)**
!python experiments/03_overfit_single_batch.py --num_samples 5 --epochs 50 --quantize

---
## Phase 4: Full Training & Evaluation 🚀 (~30-40 min)

Fine-tune BLIP-2 with LoRA on 1,793 VQA-RAD training samples.
Evaluate on 451 test samples. Compare to Med-PaLM M baselines.

**This is the main experiment. Let it run to completion.**

In [ ]:
#@title **Phase 4: Full Training Run** ⏱️ ~30-40 minutes
!python experiments/04_train_vqa.py \
    --dataset vqa_rad \
    --epochs 10 \
    --batch_size 4 \
    --lr 5e-5 \
    --lora_rank 16 \
    --grad_accum 4 \
    --quantize \
    --use_exemplar

In [ ]:
#@title **View: Training Curve**
import os, json
import matplotlib.pyplot as plt

log_path = "results/logs/training_log.json"
if os.path.exists(log_path):
    with open(log_path) as f:
        log = json.load(f)
    epochs = [e["epoch"] for e in log]
    train_loss = [e["train_loss"] for e in log]
    plt.figure(figsize=(8, 4))
    plt.plot(epochs, train_loss, 'b-o', label='Train Loss', linewidth=2)
    if "val_loss" in log[0]:
        plt.plot(epochs, [e["val_loss"] for e in log], 'r-o', label='Val Loss', linewidth=2)
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Training Progress')
    plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout()
    plt.savefig('results/figures/training_curve.png', dpi=150)
    plt.show()
    print(f"Final loss: {train_loss[-1]:.4f}")
else:
    print("⚠️ No training log — run Phase 4 first")

In [ ]:
#@title **View: Fine-Tuned Results vs Paper Baselines**
import os, json

# Check for fine-tuned results first, fall back to baseline
for path in ["results/tables/vqa_rad_metrics.json", "results/tables/baseline_metrics.json"]:
    if os.path.exists(path):
        with open(path) as f:
            results = json.load(f)
        break
else:
    print("⚠️ No results found"); results = None

if results:
    mode = results.get('mode', 'fine-tuned')
    print("=" * 60)
    print(f"RESULTS vs PAPER BASELINES (VQA-RAD) [{mode}]")
    print("=" * 60)
    rows = [
        ("Prior SOTA (specialist)",    71.03, None),
        ("PaLM-E 84B (zero-shot)",     59.19, 38.67),
        ("Med-PaLM M 12B",             64.02, 50.66),
        ("Med-PaLM M 84B",             69.38, 59.90),
        ("Med-PaLM M 562B",            71.27, 62.06),
        ("Ours (BLIP-2 + LoRA)",       results['bleu_1'], results['f1']),
    ]
    print(f"{'Model':<30} {'BLEU-1':>10} {'F1':>10}")
    print("-" * 50)
    for name, bleu, f1 in rows:
        b = f"{bleu:.2f}%" if bleu else "N/A"
        fv = f"{f1:.2f}%" if f1 else "N/A"
        marker = " ◀ OURS" if "Ours" in name else ""
        print(f"{name:<30} {b:>10} {fv:>10}{marker}")
    print("=" * 60)

---
## Phase 5: Exemplar Ablation (~15 min)

Tests the paper's claim: does the one-shot exemplar prompting trick help?

In [ ]:
#@title **Phase 5: Exemplar Ablation Study**
!python experiments/05_zero_shot_eval.py --experiment exemplar_ablation --max_samples 100 --quantize

In [ ]:
#@title **View: Exemplar Ablation Results**
import os, json

path = "results/tables/exemplar_ablation.json"
if os.path.exists(path):
    with open(path) as f:
        abl = json.load(f)
    print("ONE-SHOT EXEMPLAR ABLATION")
    print("=" * 55)
    print(f"{'Condition':<25} {'BLEU-1':>10} {'F1':>10}")
    print("-" * 45)
    for mode, m in abl.items():
        print(f"{mode:<25} {m['bleu_1']:>9.2f}% {m['f1']:>9.2f}%")
    diff = abl['with_exemplar']['bleu_1'] - abl['without_exemplar']['bleu_1']
    ratio = abl['with_exemplar']['bleu_1'] / max(abl['without_exemplar']['bleu_1'], 0.001)
    print(f"\nExemplar effect: {diff:+.2f}% BLEU-1 ({ratio:.1f}x improvement)")
    if diff > 0:
        print("✅ Confirms paper's finding: one-shot exemplars help across model scales")
    else:
        print("Interesting: exemplar did not help for our model (still a valid finding)")
else:
    print("⚠️ Run Phase 5 first")

---
## Final: Generate Comparison Report & Charts

In [ ]:
#@title **Generate Final Comparison Table + Bar Charts**
!python evaluation/compare_to_paper.py

In [ ]:
#@title **Display: Comparison Charts**
import os, glob
from IPython.display import Image, display

charts = glob.glob("results/figures/*_comparison.png")
if charts:
    for p in charts:
        display(Image(filename=p, width=700))
else:
    print("⚠️ No charts generated")

md_path = "results/tables/full_comparison.md"
if os.path.exists(md_path):
    print("\n" + open(md_path).read())

---
## Download Results

In [ ]:
#@title **Package & Download All Results**
!tar -czf /content/reproduction_results.tar.gz results/
from google.colab import files
files.download('/content/reproduction_results.tar.gz')
print("✅ Results downloaded!")

---
## Summary

| Aspect | Med-PaLM M | Our Reproduction |
|--------|-----------|------------------|
| Model | PaLM-E (562B) | BLIP-2 (~3B) |
| Training | Full fine-tuning on TPU pods | LoRA on single T4 GPU |
| Data | 1M+ samples across 14 tasks | ~3.5K VQA-RAD samples |
| Compute | Weeks on TPU v4 | ~1 hour on free Colab |

**Repository:** [github.com/Mrabbi3/biomed-multimodal-reproduction](https://github.com/Mrabbi3/biomed-multimodal-reproduction)